In [43]:
import pandas as pd
import numpy as np

In [44]:
#emp_data = pd.read_csv('data/HR_comma_sep.csv.txt')
emp_data = pd.read_csv('C:/Users/Admin/Documents/AMTH/HR_comma_sep.csv.txt')

In [45]:
emp_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [46]:
emp_data.rename(columns={'sales':'department'}, inplace=True)

In [47]:
emp_data.corr()['left']

satisfaction_level      -0.388375
last_evaluation          0.006567
number_project           0.023787
average_montly_hours     0.071287
time_spend_company       0.144822
Work_accident           -0.154622
left                     1.000000
promotion_last_5years   -0.061788
Name: left, dtype: float64

#### Selecting categorical columns & integer columns

In [48]:
cat_emp_data = emp_data.select_dtypes('object')

In [49]:
int_emp_data = emp_data.select_dtypes('int64')

#### Preprocessing Categorical Columns

In [50]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [51]:
le = LabelEncoder()
ohe = OneHotEncoder()

In [52]:
le.fit(cat_emp_data.department)

LabelEncoder()

In [53]:
cat_emp_data['department_tf'] = le.transform(cat_emp_data.department)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
cat_emp_data.head()

,department,salary,department_tf
0,sales,low,7
1,sales,medium,7
2,sales,medium,7
3,sales,low,7
4,sales,low,7


In [55]:
le.classes_

array(['IT', 'RandD', 'accounting', 'hr', 'management', 'marketing',
       'product_mng', 'sales', 'support', 'technical'], dtype=object)

In [56]:
le.inverse_transform([7])

array(['sales'], dtype=object)

In [57]:
ohe.fit(cat_emp_data[['department_tf']])

OneHotEncoder()

In [58]:
department_tf = ohe.transform(cat_emp_data[['department_tf']]).toarray()

In [59]:
from sklearn.preprocessing import FunctionTransformer

In [60]:
def func(x):
    def mapping(d):
        if d == 'low':
            return 1
        elif d == 'medium':
            return 2
        else:
            return 3
    return x.map( mapping )
        
ft = FunctionTransformer(func, validate=False)

In [61]:
cat_emp_data['salary_tf'] = ft.transform(cat_emp_data.salary)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Preprocessing Number Data

In [62]:
int_emp_data.head()

,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,2,157,3,0,1,0
1,5,262,6,0,1,0
2,7,272,4,0,1,0
3,5,223,5,0,1,0
4,2,159,3,0,1,0


In [63]:
int_emp_data.drop('left',axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [64]:
from sklearn.preprocessing import MinMaxScaler

In [65]:
mms = MinMaxScaler()

In [66]:
mms.fit(int_emp_data)

MinMaxScaler()

In [67]:
int_tf = mms.transform(int_emp_data)

In [68]:
int_tf

array([[0.        , 0.28504673, 0.125     , 0.        , 0.        ],
       [0.6       , 0.77570093, 0.5       , 0.        , 0.        ],
       [1.        , 0.82242991, 0.25      , 0.        , 0.        ],
       ...,
       [0.        , 0.21962617, 0.125     , 0.        , 0.        ],
       [0.8       , 0.85981308, 0.25      , 0.        , 0.        ],
       [0.        , 0.28971963, 0.125     , 0.        , 0.        ]])

In [69]:
float_tf = emp_data[['satisfaction_level','last_evaluation']].values

In [70]:
cat_emp_data['salary_tf'].values.shape

(14999,)

In [71]:
int_tf.shape

(14999, 5)

In [72]:
department_tf[:2]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

#### Joining the data

In [73]:
salary_tf = cat_emp_data['salary_tf'].values.reshape(-1,1)

In [74]:
feature_data = np.hstack([department_tf,int_tf,float_tf,salary_tf])

In [75]:
target_data = emp_data.left

#### Split data into two parts

In [76]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(feature_data,target_data)

#### Model Training

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [78]:
lr = LogisticRegression()

In [79]:
lr.fit(trainX,trainY)

LogisticRegression()

In [80]:
rf = RandomForestClassifier()

In [81]:
rf.fit(trainX,trainY)

RandomForestClassifier()

#### Model Validation

In [82]:
lr.score(testX,testY)

0.7864

In [83]:
rf.score(testX,testY)

0.9896

In [84]:
from sklearn.metrics import recall_score,precision_score, f1_score, classification_report

In [85]:
pred = rf.predict(testX)

In [86]:
precision_score(y_pred=pred, y_true=testY)

0.9929078014184397

In [87]:
f1_score(y_pred=pred, y_true=testY)

0.9773123909249564

In [88]:
print (classification_report(y_pred=pred, y_true=testY))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2877
           1       0.99      0.96      0.98       873

    accuracy                           0.99      3750
   macro avg       0.99      0.98      0.99      3750
weighted avg       0.99      0.99      0.99      3750

